In [1]:
from moexalgo import Ticker
import pandas as pd
import datetime

In [2]:
def get_macd(data : pd.DataFrame, short=12, long=26, signal=9) -> tuple:
    short_ema = data['close'].ewm(span=short, adjust=False).mean()
    long_ema = data['close'].ewm(span=long, adjust=False).mean()

    # Рассчитываем MACD-линию
    macd_line = short_ema - long_ema

    # Рассчитываем сигнальную линию
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()

    return macd_line, signal_line

In [3]:
def trade(data: pd.DataFrame, cash: int) -> int:
    macd, signal = get_macd(data)
    bought_price = 0
    stock_amount = 0

    for i in range(len(data)):
        if i < 28:
            continue
        else:
            if macd[i] >= signal[i] and sum([signal[j] > macd[j] for j in range(i-7, i-2)]) == 5 and stock_amount == 0:
                av_price = (data['open'][i] + data['close'][i]) / 2
                bought_price = av_price
                amount_to_buy = cash // av_price
                cash -= amount_to_buy * av_price
                stock_amount = amount_to_buy
                print(f'Куплено {amount_to_buy} акций по {av_price}, время {data['begin'][i]}')
                # buy

            if macd[i] <= macd[i-1] and stock_amount != 0:
                av_price = (data['open'][i] + data['close'][i]) / 2
                print(f'Продано {stock_amount} акций за {av_price}, время {data['begin'][i]}')
                cash += av_price * stock_amount
                stock_amount = 0
                # sell
        
        if i == len(data) - 1 and stock_amount != 0:
            av_price = (data['open'][i] + data['close'][i]) / 2
            print(f'Продано {stock_amount} акций за {av_price}, время {data['begin'][i]}')
            cash += av_price * stock_amount
            stock_amount = 0

    return cash

In [4]:
ticker = Ticker('POLY')
curr_date = datetime.date.today()
delta = curr_date - datetime.timedelta(weeks=12)
curr_date, delta = str(curr_date), str(delta)

In [5]:
candles = ticker.candles(period='1m', date=delta, till_date=curr_date)

In [6]:
trade(candles, 20000)

Куплено 35.0 акций по 569.5, время 2023-09-19 11:09:00
Продано 35.0 акций за 571.8, время 2023-09-19 11:24:00
Куплено 35.0 акций по 571.35, время 2023-09-19 11:37:00
Продано 35.0 акций за 573.05, время 2023-09-19 11:46:00
Куплено 35.0 акций по 565.6, время 2023-09-19 12:27:00
Продано 35.0 акций за 566.75, время 2023-09-19 12:39:00
Куплено 36.0 акций по 555.8, время 2023-09-19 12:59:00
Продано 36.0 акций за 554.45, время 2023-09-19 13:06:00
Куплено 35.0 акций по 566.9, время 2023-09-19 14:13:00
Продано 35.0 акций за 565.8, время 2023-09-19 14:15:00
Куплено 35.0 акций по 560.75, время 2023-09-19 14:37:00
Продано 35.0 акций за 562.05, время 2023-09-19 14:47:00
Куплено 36.0 акций по 556.75, время 2023-09-19 15:15:00
Продано 36.0 акций за 556.15, время 2023-09-19 15:19:00
Куплено 36.0 акций по 552.1500000000001, время 2023-09-19 15:28:00
Продано 36.0 акций за 550.0, время 2023-09-19 15:33:00
Куплено 36.0 акций по 548.9, время 2023-09-19 15:41:00
Продано 36.0 акций за 550.5, время 2023-09-19

24343.150000000045